### 분석 데이터 배경
- 데이터셋은 영화 27278개의 20000263 레이팅과 465564의 태깅으로 구성
- 1995년부터 2015년까지 138493의 유저들이 입력한 데이터로 구성
- 각 유저들은 최소 20개의 영화에 대한 레이팅을 진행했음

### 분석 데이터 설명
- 개인 정보 이슈로 인해서 개인 데모 정보는 포함되어 있지 않음
- 모든 유저들은 id로 표현이 되며 그 외의 정보는 제공되지 않음

### 데이터는 총 6가지 테이블로 구성

1. tag.csv: 유저들이 영화마다 적용한 태그 정보

- userId, movieId, tag, timestamp

2. rating.csv : 유저들이 매겨놓은 영화 레이팅 정보

- userId, movieId, rating, timestamp

3. movie.csv: 영화 정보

- movieId, title, genres

4. link.csv: 다른 정보들과 연계해서 분석할 수 있는 매핑 테이블

- movieId, imdbId, tmbdId

5. genome_scores.csv: 태그 정보와 관련된 정보가 들어있는 테이블

- movieId, tagId, relevance

6. genome_tags.csv: 태그 정보 매핑 테이블

- tagId, tag

##### 분석 참고 링크: https://www.kaggle.com/anmol4210/recommendation-system-collaborative-filtering

### 1. 패키지, 데이터 셋 호출

In [61]:
## 패키지 호출
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
## 데이터 경로 지정
path= 'data/'
movie= pd.read_csv(path+'movie.csv')
rating= pd.read_csv(path+'rating.csv')
genome_score= pd.read_csv(path+'genome_scores.csv')
link= pd.read_csv(path+'link.csv')
genome_tag= pd.read_csv(path+'genome_tags.csv')

### 2. EDA

In [24]:
## 전체 데이터 살펴보기
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
len(movie)

27278

In [26]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [27]:
len(rating)

20000263

In [28]:
genome_score.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [29]:
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [30]:
genome_tag.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


### 2. 데이터 정제

In [37]:
## 무비 데이터 내 제목 컬럼에서 제목, 연도 컬럼 추출

## 연도 컬럼 추출
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))',expand=False)
movie['year'] = movie.year.str.extract('(\d\d\d\d)',expand=False)

## 제목 컬럼 추출
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
movie['title'] = movie['title'].apply(lambda x: x.strip())

In [38]:
movie.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [39]:
## 장르 컬럼 제거
movie.drop(columns=['genres'],inplace=True)

In [40]:
## rating 테이블에서 genre 컬럼 제거
rating.drop(columns=['timestamp'],inplace=True)

### 3. Collaborative Filtering 방법론 적용

- 다른 유저의 정보를 사용해서 새로운 유저들에게 아이템을 추천하는 방법
- 비슷한 취향을 가진 유저들을 찾아서 새로운 유저에게 기존에 비슷한 취향을 가진 유저들의 아이템들을 추천
- 머신 러닝 등 다양한 분석 방법론이 있지만, 여기서는 Pearson Correlation Function을 사용할 예정

#### 유저 베이스 추천 시스템의 프로세스
1. 유저를 선정 (본 영화를 베이스로)
2. 영화 레이팅을 기반으로, 가장 유사한 취향을 가진 유저를 찾기
3. 가장 유사한 취향을 가진 유저들이 봤던 영화를 가져오기
4. similarity score 계산
5. similarity score의 점수가 가장 높은 아이템 순으로 추천

In [43]:
## test input 데이터 가져오기
user = [
            {'title':'Breakfast Club, The', 'rating':4},
            {'title':'Toy Story', 'rating':2.5},
            {'title':'Jumanji', 'rating':3},
            {'title':"Pulp Fiction", 'rating':4.5},
            {'title':'Akira', 'rating':5}
         ] 
inputMovie = pd.DataFrame(user)
inputMovie

,title,rating
0,"Breakfast Club, The",4.0
1,Toy Story,2.5
2,Jumanji,3.0
3,Pulp Fiction,4.5
4,Akira,5.0


In [46]:
# 무비 id 정보 붙이기 (불필요한 연도 데이터 제거)
Id = movie[movie['title'].isin(inputMovie['title'].tolist())]
inputMovie = pd.merge(Id, inputMovie)
inputMovie = inputMovie.drop('year', 1)
inputMovie

,movieId,title,rating
0,1,Toy Story,2.5
1,2,Jumanji,3.0
2,296,Pulp Fiction,4.5
3,1274,Akira,5.0
4,1968,"Breakfast Club, The",4.0


In [45]:
Id

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
293,296,Pulp Fiction,1994
1246,1274,Akira,1988
1884,1968,"Breakfast Club, The",1985


In [49]:
## 레이팅 테이블 내에서 해당 영화를 봤던 유저들만 선정하기
users = rating[rating['movieId'].isin(inputMovie['movieId'].tolist())]
users.head()

,userId,movieId,rating
0,1,2,3.5
11,1,296,4.0
236,3,1,4.0
451,5,2,3.0
517,6,1,5.0


In [50]:
users.shape

(168730, 3)

In [52]:
## user subset 그룹 구하기
user_subset= users.groupby(['userId'])

In [54]:
## 샘플 user에 대한 정보
user_subset.get_group(1130)

,userId,movieId,rating
166633,1130,1968,4.0


In [55]:
# 가장 비슷한 영화를 본 순서대로 유저들 정렬하기
userSubsetGroup = sorted(user_subset,  key=lambda x: len(x[1]), reverse=True)

In [58]:
userSubsetGroup[0:3]

[(91,
        userId  movieId  rating
  9621      91        1     4.0
  9622      91        2     3.5
  9669      91      296     3.5
  9826      91     1274     2.5
  9903      91     1968     4.0),
 (294,
         userId  movieId  rating
  37452     294        1     4.5
  37453     294        2     4.5
  37504     294      296     4.5
  37648     294     1274     4.5
  37731     294     1968     5.0),
 (586,
         userId  movieId  rating
  81164     586        1     2.5
  81165     586        2     3.0
  81226     586      296     5.0
  81390     586     1274     4.0
  81499     586     1968     3.0)]

### Pearson correlation 산출

- Pearson 상관은 두 계량형 변수 사이의 선형 관계를 평가
- Pearson 상관값(r)은 -1 ~ 1의 범위를 가지고 있음

In [59]:
userSubsetGroup = userSubsetGroup[0:100]

In [62]:
## Pearson 상관 계수를 저장해놓을 Dict
pearsonCorDict = {}

## For loop 실행하면서 각 유저들 간의 상관계수값 산출
for name, group in userSubsetGroup:
    # 현재 유저 그룹과 테스트 유저 데이터가 나중에 섞이지 않도록 변수 분리
    group = group.sort_values(by='movieId')
    inputMovie = inputMovie.sort_values(by='movieId')
    n = len(group)
    # 기존 유저 그룹과 테스트 유저 둘 다 본 영화 리뷰 스코어 가져오기
    temp = inputMovie[inputMovie['movieId'].isin(group['movieId'].tolist())]
    # 템프 레이팅 리스트 생성
    tempRatingList = temp['rating'].tolist()
    # 현재 유저 그룹들의 리뷰를 리스트 포맷으로 생성
    tempGroupList = group['rating'].tolist()
    
    # 현재 유저 그룹과 테스트 유저 간의 pearson 상관계수값 산출
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
    
    if Sxx != 0 and Syy != 0:
        pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorDict[name] = 0

In [64]:
## 상관계수값 보기
pearsonCorDict.items()

dict_items([(91, -0.6890618270883883), (294, 0.10783277320343156), (586, 0.7836445860269199), (648, 0.444102681159703), (775, 0.46266531814837414), (812, -0.12945217625467967), (869, 0.07624928516630236), (903, 0.0660338179744212), (1200, 0.2494610901255917), (1244, 0.29654012630945475), (1715, 0.6309898162000303), (1748, 0.5114957546028552), (1763, 0.1760901812651271), (1810, 0.6990252954195334), (1813, 0.36589645615870564), (1849, 0.06603381797442423), (1864, 0.5114957546028552), (1942, 0.23262521394079627), (1984, -0.7994259492812168), (2047, 0.5477103564747346), (2099, -0.10783277320343156), (2367, -0.10783277320343994), (2397, 0), (2515, 0.9244734516419062), (2661, 0.835703992326648), (2757, 0.8439249387982215), (2959, 0.23055616708169688), (2988, 0.29809064964264287), (3179, 0.0), (3218, 0.26413527189768793), (3268, 0.7781270639007126), (3269, 0.3606167767094639), (3318, 0.3026049692947228), (3397, 0.46107317554294897), (3487, -0.3774147062120338), (3576, 0.39620290784652895), (3

In [65]:
## pearson 상관계수를 데이터프레임 형태로 변환
pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,-0.689062,91
1,0.107833,294
2,0.783645,586
3,0.444103,648
4,0.462665,775


In [66]:
## Pearson 상관 계수가 높은 top user들 선정 (50명)
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
77,0.964486,9305
80,0.951044,9650
63,0.940064,8032
81,0.924473,9772
23,0.924473,2515


In [67]:
## 가장 유사한 50명의 유저들의 레이팅 정보 조인
topUsersRating=topUsers.merge(rating, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.964486,9305,1,1.0
1,0.964486,9305,2,1.0
2,0.964486,9305,11,1.0
3,0.964486,9305,21,1.0
4,0.964486,9305,32,2.0


In [68]:
# 유저 레이팅 점수와 유사도를 곱해서 가중 레이팅 점수를 산출
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.964486,9305,1,1.0,0.964486
1,0.964486,9305,2,1.0,0.964486
2,0.964486,9305,11,1.0,0.964486
3,0.964486,9305,21,1.0,0.964486
4,0.964486,9305,32,2.0,1.928971
